## Generate retrospective forecasts using NeuralProphet

Use NeuralProphet to generate retrospective forecasts.

In [1]:
import pandas as pd
pd.set_option('precision', 3)

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import os

from neuralprophet import NeuralProphet
from sklearn.model_selection import ParameterGrid


In [2]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

year_period = {'MS': 12, 'W': 52, 'D': 365}
frequency = 'MS'
one_year = year_period[frequency]

## Load Data From Files
Be sure to run `load_data.ipynb` to produce the data file `all_data.csv`

In [3]:
foodprice_categories = pd.read_csv("./foodprice_categories.txt", sep='\n', header=None)[0].to_list()
foodprice_df = pd.read_csv("./all_data.csv.bak", index_col=0)
foodprice_df = foodprice_df.set_index(pd.DatetimeIndex(foodprice_df.index))
foodprice_df

,Bakery and cereal products (excluding baby food),Dairy products and eggs,"Fish, seafood and other marine products",Food purchased from restaurants,Food,"Fruit, fruit preparations and nuts",Meat,Other food products and non-alcoholic beverages,Vegetables and vegetable preparations,DEXCAUS,...,XTIMVA01CAM657S,XTIMVA01CAM659S,XTIMVA01CAM664N,XTIMVA01CAM664S,XTIMVA01CAM667S,XTNTVA01CAM664N,XTNTVA01CAM664S,XTNTVA01CAM667S,TOTALNS,TOTALSL
1986-01-01,69.3,70.9,60.6,59.1,67.3,76.0,65.1,77.5,76.0,1.392,...,3.644,12.051,9.368e+09,9.496e+09,6.749e+09,6.906e+08,1.052e+09,7.479e+08,607.369,605.703
1986-02-01,70.3,70.8,61.3,59.1,66.9,77.6,64.2,78.1,68.4,1.392,...,1.965,16.745,9.495e+09,9.632e+09,6.881e+09,-9.880e+07,1.539e+08,1.099e+08,605.807,610.678
1986-03-01,70.6,71.1,61.3,59.3,67.0,79.2,64.2,78.6,66.2,1.392,...,-11.565,1.655,8.803e+09,8.529e+09,6.085e+09,9.138e+08,9.079e+08,6.478e+08,606.799,613.377
1986-04-01,71.3,71.0,61.4,59.7,67.7,82.2,63.6,79.5,71.1,1.392,...,13.334,10.821,1.034e+10,9.569e+09,6.897e+09,3.470e+08,6.563e+08,4.730e+08,614.367,619.658
1986-05-01,71.2,71.4,61.9,59.9,68.2,83.5,64.0,79.8,75.3,1.377,...,-4.236,6.160,9.598e+09,9.091e+09,6.605e+09,7.013e+08,6.893e+08,5.008e+08,621.915,625.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,157.7,145.3,146.2,163.9,156.8,144.5,176.7,142.2,153.4,1.203,...,-1.087,31.561,5.159e+10,4.979e+10,4.075e+10,3.661e+09,3.111e+09,2.546e+09,4259.604,4307.137
2021-07-01,157.9,146.4,146.6,165.2,157.6,141.7,180.9,141.9,154.8,1.240,...,1.990,18.580,5.106e+10,5.205e+10,4.156e+10,3.780e+08,1.129e+09,9.018e+08,4276.202,4323.730
2021-08-01,158.5,148.3,146.8,165.9,158.0,142.5,182.1,141.7,152.2,1.251,...,-2.312,14.723,5.245e+10,5.115e+10,4.060e+10,1.300e+09,2.674e+09,2.123e+09,4316.303,4337.489
2021-09-01,158.1,148.0,147.1,165.9,158.5,141.5,184.8,144.3,150.0,1.262,...,-3.461,8.557,5.080e+10,4.965e+10,3.920e+10,1.561e+09,2.277e+09,1.798e+09,4346.662,4367.402


## Experiment Setup

In [4]:
report_sim_dates = pd.read_csv("./reportsimdates.txt", sep='\n', header=None)[0].to_list()
report_sim_dates

['2015-07-01',
 '2016-07-01',
 '2017-07-01',
 '2018-07-01',
 '2019-07-01',
 '2020-07-01']

In [5]:
sim_train_dates = {}
sim_valid_dates = {}

for date in report_sim_dates:
    sim_train_dates[date] = foodprice_df.index[foodprice_df.index <= date]
    sim_valid_dates[date] = foodprice_df.index[(foodprice_df.index > date) & (foodprice_df.index <= (pd.to_datetime(date) + pd.DateOffset(months=18)))]

In [6]:
dataset_df = foodprice_df[foodprice_categories].T
dataset_df

,1986-01-01,1986-02-01,1986-03-01,1986-04-01,1986-05-01,1986-06-01,1986-07-01,1986-08-01,1986-09-01,1986-10-01,...,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01
Bakery and cereal products (excluding baby food),69.3,70.3,70.6,71.3,71.2,71.1,71.7,71.9,71.7,71.1,...,154.2,157.1,156.8,156.2,157.8,157.7,157.9,158.5,158.1,159.3
Dairy products and eggs,70.9,70.8,71.1,71.0,71.4,71.1,71.3,71.5,71.8,71.8,...,141.6,143.1,144.9,146.1,146.6,145.3,146.4,148.3,148.0,146.6
"Fish, seafood and other marine products",60.6,61.3,61.3,61.4,61.9,62.0,62.2,62.7,63.1,63.6,...,143.4,143.9,144.9,145.1,147.6,146.2,146.6,146.8,147.1,149.3
Food purchased from restaurants,59.1,59.1,59.3,59.7,59.9,60.0,60.6,60.9,60.9,61.3,...,162.6,162.9,162.6,163.2,163.5,163.9,165.2,165.9,165.9,167.1
Food,67.3,66.9,67.0,67.7,68.2,68.4,69.2,69.5,69.9,70.2,...,155.0,155.6,155.5,155.4,156.6,156.8,157.6,158.0,158.5,159.2
"Fruit, fruit preparations and nuts",76.0,77.6,79.2,82.2,83.5,83.1,84.8,86.7,83.8,82.9,...,140.9,143.4,142.4,141.9,143.9,144.5,141.7,142.5,141.5,141.7
Meat,65.1,64.2,64.2,63.6,64.0,64.9,66.5,67.8,71.3,71.5,...,171.9,169.5,170.2,173.5,175.4,176.7,180.9,182.1,184.8,187.1
Other food products and non-alcoholic beverages,77.5,78.1,78.6,79.5,79.8,79.9,80.2,80.2,80.8,81.0,...,139.0,139.5,141.5,140.5,141.6,142.2,141.9,141.7,144.3,144.4
Vegetables and vegetable preparations,76.0,68.4,66.2,71.1,75.3,74.1,75.7,71.9,66.6,70.7,...,162.5,163.8,157.4,151.1,153.8,153.4,154.8,152.2,150.0,149.4


In [7]:
def get_neuralprophet_df(foodprice_df, food_category, dates, lagged_regressor):
    prophet_df = pd.DataFrame({'ds': foodprice_df[food_category][dates].index, 
                           'y':foodprice_df[food_category].loc[foodprice_df.index][dates], 
                           lagged_regressor: foodprice_df[lagged_regressor].loc[foodprice_df.index][dates],
                          })
    prophet_df.reset_index(drop=True, inplace=True)

    return prophet_df
    

In [8]:
data_sources = ["DEXCAUS",
                "DCOILWTICO",
                "WILL5000IND",
                "VXOCLS",
                "CUSR0000SAF112",
                "CUSR0000SAF113",
                "CPIFABSL",
                "UNRATE",
                "FEDFUNDS",
                "IRLTLT01CAM156N",
                "LRUNTTTTCAM156S",
                "CPALCY01CAM661N",
                "CPGRLE01CAM657N",
                "QCAR368BIS"
               ]

In [9]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = {
    'r2_score': r2_score,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error,
    'mse': mean_squared_error,
    'rmse': rmse
}



## Fit Models and Evaluate Models for All CPI Categories

Fit a model using data up to the `report_sim_date` and use it to produce forecasts for the following 18 months. Collect evaluation metrics for comparison and save the forecasts for use in the ensembling experiments.

Use ParameterGrid for Hyperparameter tuning.

In [11]:
params_grid = {'n_forecasts':[18],
              'n_lags':[24, 36, 48, 60]}

# TESTING WITH REDUCED PARAMS
params_grid = {'n_forecasts':[18],
              'n_lags':[24]}
data_sources = ['DEXCAUS']

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1
print('Total Possible Models',cnt)

Total Possible Models 1


In [12]:
all_output_paths = []
all_valid_metrics = {}
all_forecasts = {}
for var in data_sources:
    for p in grid:
        details = str(var) + '_nlags_' + str(p['n_lags'])
        output_path = './output/neuralprophet_TEST_' + details
        if not os.path.exists(output_path):
            print("new directory created: ", output_path)
            os.mkdir(output_path)
        else:
            print("directory already exists, skipping")
            continue
        all_output_paths.append(output_path)
        
        
        for food_category in foodprice_df.columns[:9]:
            
            annual_valid_metrics = {}
            annual_forecasts = {}
                        
            for date in report_sim_dates:
                
                train_dates = sim_train_dates[date]
                valid_dates = sim_valid_dates[date]
                
                
                train_model =NeuralProphet(n_forecasts = p['n_forecasts'],                          #initialize model
                                           n_lags = p['n_lags'])
                train_df = get_neuralprophet_df(foodprice_df, food_category, train_dates, str(var)) #select data
                train_model = train_model.add_lagged_regressor(names=[str(var)])                   #lagged regressor to model
                train_model.fit(train_df, freq='MS')                                              #fit the model
                
                
                future = train_model.make_future_dataframe(train_df, periods=18)                #create the future period
                forecast = train_model.predict(future)                                        #produce the future period forecast
                
                
                all_yhats = []
                for yhat in ["yhat" + str(i) for i in range(1,19)]:
                    all_yhats.append((forecast.loc[forecast[yhat].first_valid_index()][yhat]))  #get all yhats from forecast

                
                valid_df = get_neuralprophet_df(foodprice_df, food_category, valid_dates, str(var))
                
                ##special case for 2020 forecast
                if len(valid_df) == 14:
                    all_yhats = all_yhats[:-4]
                    
                ##special case
                if len(valid_df) == 15:
                    all_yhats = all_yhats[:-3]
                
                ## compute valid metrics
                valid_metrics = {metric_name: metric_fn(y_true=valid_df.y, y_pred=all_yhats) for metric_name, metric_fn in metrics.items()}
                valid_metrics_series = pd.Series(valid_metrics)
                
                annual_valid_metrics[date] = pd.DataFrame(valid_metrics_series).mean(axis=1)
                annual_forecasts[date] = pd.Series(all_yhats)
                            
            
                # break ### loop for dates 
                
            all_valid_metrics[(output_path, food_category)] = annual_valid_metrics
            all_forecasts[(output_path, food_category)] = annual_forecasts
                
            mean_valid_metrics = {}
            for (output_path,food_category), valid_scores in all_valid_metrics.items():
                mean_valid_metrics[food_category] = pd.DataFrame(valid_scores).mean(axis=1)

            mean_valid_metrics_df = pd.DataFrame(mean_valid_metrics).T
            mean_valid_metrics_df.to_csv(f"{output_path}/mean_fc_valid_metrics.csv")
        
            # break ## loop for food category
        
        # break ## loop for n_lags

    # break ##loop for lagged regressors



INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 233


  0%|          | 0/225 [00:00<?, ?it/s]

new directory created:  ./output/neuralprophet_TEST_DEXCAUS_nlags_24


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.63E-02, min: 4.26E-01


  0%|          | 0/225 [00:00<?, ?it/s]

...

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.83E-02, min: 3.91E-01


  0%|          | 0/229 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.29E-02, min: 4.28E-01


  0%|          | 0/229 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.41E-02, min: 4.28E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 4.83E-02
Epoch[225/225]: 100%|██████████| 225/225 [00:16<00:00, 13.84it/s, SmoothL1Loss=0.00124, MAE=3.11, RMSE=4.06, RegLoss=0]


## Save Validation Forecasts to File (For Ensembling)


In [13]:
date_forecasts = {}
for date in report_sim_dates:
    valid_dates = sim_valid_dates[date]
    food_forecasts = {}
    for my_output_path in all_output_paths:
        for food_category in foodprice_categories:
            food_forecasts[food_category] = all_forecasts[(my_output_path,food_category)][date]
        date_forecasts[date] = pd.DataFrame(food_forecasts)
        date_forecasts[date] = date_forecasts[date].set_index(pd.DatetimeIndex(valid_dates))
        date_forecasts[date].to_csv(f"{my_output_path}/forecasts_{date}.csv")